# Load datasets from google drive #

In [1]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!mkdir "/content/images"

!unzip "/content/drive/MyDrive/topography_dataset/all_10k.zip" -d "/content/images"
!unzip "/content/drive/MyDrive/topography_dataset/average_10k.zip" -d "/content/images"
!unzip "/content/drive/MyDrive/topography_dataset/blackbox_10k.zip" -d "/content/images"
!unzip "/content/drive/MyDrive/topography_dataset/bubble_10k.zip" -d "/content/images"
!unzip "/content/drive/MyDrive/topography_dataset/pizza_10k.zip" -d "/content/images"
!unzip "/content/drive/MyDrive/topography_dataset/pure_10k.zip" -d "/content/images"

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
  inflating: /content/images/pure_10k/pure_05001.png  
  inflating: /content/images/pure_10k/pure_05002.png  
  inflating: /content/images/pure_10k/pure_05003.png  
  inflating: /content/images/pure_10k/pure_05004.png  
  inflating: /content/images/pure_10k/pure_05005.png  
  inflating: /content/images/pure_10k/pure_05006.png  
  inflating: /content/images/pure_10k/pure_05007.png  
  inflating: /content/images/pure_10k/pure_05008.png  
  inflating: /content/images/pure_10k/pure_05009.png  
  inflating: /content/images/pure_10k/pure_05010.png  
  inflating: /content/images/pure_10k/pure_05011.png  
  inflating: /content/images/pure_10k/pure_05012.png  
  inflating: /content/images/pure_10k/pure_05013.png  
  inflating: /content/images/pure_10k/pure_05014.png  
  inflating: /content/images/pure_10k/pure_05015.png  
  inflating: /content/images/pure_10k/pure_05016.png  
  inflating: /content/images/pure_10k/pure_05017.png  


In [1]:
import os
import shutil

main_dir = "images"
img_dirs = [
    # "all_10k",
    "average_10k",
    # "blackbox_10k",
    # "bubble_10k",
    # "pizza_10k",
    # "pure_10k"
]

for name in img_dirs:
    photos_dir = os.path.join(main_dir, name)
    # List all files in the "photos" directory
    photo_files = os.listdir(photos_dir)

    # Move each photo file to the "main" directory
    for photo_file in photo_files:
        src_path = os.path.join(photos_dir, photo_file)
        dest_path = os.path.join(main_dir, photo_file)
        shutil.move(src_path, dest_path)

    # Remove the "photos" directory
    os.rmdir(photos_dir)

In [4]:
!mkdir params

!cp /content/drive/MyDrive/topography_dataset/*.csv /content/params

# Pytorch dataloaders and datasets #

In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

import numpy as np
import pandas as pd

from PIL import Image

from sklearn.model_selection import train_test_split

In [2]:
class ImageDataset(Dataset):
    def __init__(self, path: str, parameters: pd.DataFrame, transform=None) -> None:
        self.image_paths = np.array(
            [path + filename for filename in parameters["filename"].to_numpy()]
        )
        self.labels = parameters["epsilon"].to_numpy()
        self.transform = transform

    def __getitem__(self, inx):
        image_path = self.image_paths[inx]
        label_float_epsilon = self.labels[inx]
        target = np.zeros(shape=(1000,))
        target[int(label_float_epsilon * 1000)] = 1
        image = Image.open(image_path)
        image = np.array(image)
        # repeat grayscale value three times for all RGB channels
        image = np.repeat(image[..., np.newaxis], 3, -1)
        if self.transform:
            image = self.transform(image)
        return image, torch.from_numpy(target)

    def __len__(self):
        return len(self.image_paths)

In [3]:
transform = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5]),
    ]
)

## Split train dataframes into training and validation ##

Train: 6000 <br>
Valid: 1000

In [4]:
main_params_dir = "params/"

In [5]:
all_csv = pd.read_csv(main_params_dir + "all_train.csv", index_col=False)
all_train, all_valid = train_test_split(
    all_csv, test_size=1000, random_state=12, shuffle=True, stratify=all_csv["epsilon"]
)

average_csv = pd.read_csv(main_params_dir + "average_train.csv", index_col=False)
average_train, average_valid = train_test_split(
    average_csv,
    test_size=1000,
    random_state=12,
    shuffle=True,
    stratify=average_csv["epsilon"],
)

blackbox_csv = pd.read_csv(main_params_dir + "blackbox_train.csv", index_col=False)
blackbox_train, blackbox_valid = train_test_split(
    blackbox_csv,
    test_size=1000,
    random_state=12,
    shuffle=True,
    stratify=blackbox_csv["epsilon"],
)

bubble_csv = pd.read_csv(main_params_dir + "bubble_train.csv", index_col=False)
bubble_train, bubble_valid = train_test_split(
    bubble_csv,
    test_size=1000,
    random_state=12,
    shuffle=True,
    stratify=bubble_csv["epsilon"],
)

pizza_csv = pd.read_csv(main_params_dir + "pizza_train.csv", index_col=False)
pizza_train, pizza_valid = train_test_split(
    pizza_csv,
    test_size=1000,
    random_state=12,
    shuffle=True,
    stratify=pizza_csv["epsilon"],
)

pure_csv = pd.read_csv(main_params_dir + "pure_train.csv", index_col=False)
pure_train, pure_valid = train_test_split(
    pure_csv,
    test_size=1000,
    random_state=12,
    shuffle=True,
    stratify=pure_csv["epsilon"],
)

In [6]:
DATASETS = {
    "all": {
        "train": all_train,
        "valid": all_valid,
        "test": pd.read_csv(main_params_dir + "all_test.csv", index_col=False),
    },
    "average": {
        "train": average_train,
        "valid": average_valid,
        "test": pd.read_csv(main_params_dir + "average_test.csv", index_col=False),
    },
    "blackbox": {
        "train": blackbox_train,
        "valid": blackbox_valid,
        "test": pd.read_csv(main_params_dir + "blackbox_test.csv", index_col=False),
    },
    "bubble": {
        "train": bubble_train,
        "valid": bubble_valid,
        "test": pd.read_csv(main_params_dir + "bubble_test.csv", index_col=False),
    },
    "pizza": {
        "train": pizza_train,
        "valid": pizza_valid,
        "test": pd.read_csv(main_params_dir + "pizza_test.csv", index_col=False),
    },
    "pure": {
        "train": pure_train,
        "valid": pure_valid,
        "test": pd.read_csv(main_params_dir + "pure_test.csv", index_col=False),
    },
}

# Model training #

We will train 7 models, each will be train on one choosen train dataset (6 models) and there will be extra trained model on all datasets together. <br>

Each model will be evaluated on all test sets separately. <br>

Each model will have the same architecture based on ResNet18. <br>

In results, there will be one result table for each model.

In [7]:
import torch.nn as nn
from torchvision import models
import torch.optim as optim
from tqdm import tqdm

In [8]:
device = "cuda" if torch.cuda.is_available else "cpu"
lr = 1e-4
epochs = 5
main_dir = "images/"
print(device)

cuda


In [10]:
def cyclic_MAE(outputs, targets):
    x1 = torch.minimum(
        torch.abs(outputs - targets),
        torch.minimum(
            torch.abs(outputs - targets + 1), torch.abs(outputs - targets - 1)
        ),
    )
    return torch.mean(x1)

In [9]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

## Pure Model - model trained using only pure image. ##

In [21]:
train_dataset = ImageDataset(
    path=main_dir, parameters=DATASETS["all"]["train"], transform=transform
)
valid_dataset = ImageDataset(
    path=main_dir, parameters=DATASETS["all"]["valid"], transform=transform
)
test_dataset = ImageDataset(
    path=main_dir, parameters=DATASETS["all"]["test"], transform=transform
)

In [22]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True, num_workers=0)

In [30]:
model = models.resnet18(pretrained=True)

classifier = nn.Sequential(
    nn.Linear(512, 512), nn.ReLU(), nn.Dropout(0.3), nn.ReLU(), nn.Linear(512, 1000)
)

model.fc = classifier

/home/ute/users/danielj/topography-surface/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ute/users/danielj/topography-surface/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [31]:
optimizer = torch.optim.AdamW(params=model.parameters(), lr=lr, eps=1e-8)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, patience=1, factor=0.9, min_lr=1e-5
)

In [32]:
min_valid_loss = np.inf
history = {"train_losses": [], "valid_losses": []}

model.cuda()

for epoch in tqdm(range(epochs)):
    model.train()
    train_batch_losses = []
    for data, labels in train_loader:
        model.zero_grad()
        optimizer.zero_grad()
        images = data.cuda()
        targets = labels.cuda()
        outputs = model(images)

        loss = loss_fn(outputs, targets)

        # optimizer.zero_grad()
        loss.backward()
        # torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_batch_losses.append(loss.item())
    train_loss = np.sum(train_batch_losses) / len(train_batch_losses)
    history["train_losses"].append(train_loss)

    model.eval()
    fin_targets = []
    fin_outputs = []
    valid_batch_losses = []
    with torch.no_grad():
        for data, labels in valid_loader:
            images = data.cuda()
            targets = labels.cuda()
            outputs = model(images)

            loss = loss_fn(outputs, targets)
            valid_batch_losses.append(loss.item())
        valid_loss = np.sum(valid_batch_losses) / len(valid_batch_losses)
        history["valid_losses"].append(valid_loss)

    if min_valid_loss > valid_loss:
        torch.save(model.state_dict(), "best_model_pure.pth")
        min_valid_loss = valid_loss

    print(
        f"Epoch {epoch} \t\t Training Loss: {train_loss} \t\t Validation Loss: {valid_loss}"
    )
    scheduler.step(valid_loss)

    torch.save(model.state_dict(), "final_model_pure.pth")

  0%|          | 0/5 [00:00<?, ?it/s]

 20%|██        | 1/5 [04:03<16:14, 243.74s/it]

Epoch 0 		 Training Loss: 0.05931281087539416 		 Validation Loss: 0.008063755772596173


 40%|████      | 2/5 [08:10<12:17, 245.74s/it]

Epoch 1 		 Training Loss: 0.008740754767992635 		 Validation Loss: 0.008008862388877483


...

Epoch 2 		 Training Loss: 0.008638487120891063 		 Validation Loss: 0.007978677284122786


 80%|████████  | 4/5 [16:17<04:03, 243.82s/it]

Epoch 3 		 Training Loss: 0.008538941055537033 		 Validation Loss: 0.007918999291881789


100%|██████████| 5/5 [20:21<00:00, 244.30s/it]

Epoch 4 		 Training Loss: 0.008296974544163914 		 Validation Loss: 0.007426297478977161


In [33]:
def test(test_loader):
    model.eval()
    fin_targets = []
    fin_outputs = []
    with torch.no_grad():
        for data, labels in tqdm(test_loader):
            images = data.cuda()
            targets = labels.cuda()
            outputs = model(images)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [34]:
def cyclic_MAE_test(output, target):
    return min(
        abs(output - target), min(abs(output - target + 1), abs(output - target - 1))
    )

In [35]:
def save_results(outputs, targets, filename: str):
    epsilons = []
    predicted = []
    differences = []
    for out, tar in zip(outputs, targets):
        epsilons.append(tar.index(1) / 1000)
        predicted.append(out.index(1) / 1000)
        differences.append(cyclic_MAE_test(out.index(1) / 1000, tar.index(1) / 1000))
    results_df = pd.DataFrame(
        list(zip(epsilons, predicted, differences)),
        columns=["epsilon", "predicted", "loss"],
    )
    results_df.to_csv(filename, index=False)
    return results_df

### Loss on pure images ###

In [36]:
outputs, targets = test(test_loader)

outputs_numpy = np.array(outputs)
out = np.zeros_like(outputs_numpy)
out[np.arange(len(outputs_numpy)), outputs_numpy.argmax(1)] = 1
out = out.tolist()
results = save_results(out, targets, "pure_model-pure_noise_results.csv")
results["loss"].mean()

100%|██████████| 188/188 [01:13<00:00,  2.55it/s]


0.05206333333333334

### Loss on average-noise images

In [37]:
test_dataset = ImageDataset(
    path=main_dir, parameters=DATASETS["average"]["test"], transform=transform
)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True, num_workers=0)

outputs, targets = test(test_loader)

outputs_numpy = np.array(outputs)
out = np.zeros_like(outputs_numpy)
out[np.arange(len(outputs_numpy)), outputs_numpy.argmax(1)] = 1
out = out.tolist()
results = save_results(out, targets, "pure_model-average_noise_results.csv")
results["loss"].mean()

100%|██████████| 188/188 [01:21<00:00,  2.30it/s]


0.25

### Loss on all-noise images

In [39]:
test_dataset = ImageDataset(
    path=main_dir, parameters=DATASETS["all"]["test"], transform=transform
)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True, num_workers=0)

outputs, targets = test(test_loader)

outputs_numpy = np.array(outputs)
out = np.zeros_like(outputs_numpy)
out[np.arange(len(outputs_numpy)), outputs_numpy.argmax(1)] = 1
out = out.tolist()
results = save_results(out, targets, "pure_model-all_noise_results.csv")
results["loss"].mean()

100%|██████████| 188/188 [01:19<00:00,  2.37it/s]


0.2500026666666667

### Loss on blackbox-noise images

In [38]:
test_dataset = ImageDataset(
    path=main_dir, parameters=DATASETS["blackbox"]["test"], transform=transform
)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True, num_workers=0)

outputs, targets = test(test_loader)

outputs_numpy = np.array(outputs)
out = np.zeros_like(outputs_numpy)
out[np.arange(len(outputs_numpy)), outputs_numpy.argmax(1)] = 1
out = out.tolist()
results = save_results(out, targets, "pure_model-blackbox_noise_results.csv")
results["loss"].mean()

100%|██████████| 188/188 [01:17<00:00,  2.44it/s]


0.05410633333333334

### Loss on bubble-noise images

In [40]:
test_dataset = ImageDataset(
    path=main_dir, parameters=DATASETS["bubble"]["test"], transform=transform
)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True, num_workers=0)

outputs, targets = test(test_loader)

outputs_numpy = np.array(outputs)
out = np.zeros_like(outputs_numpy)
out[np.arange(len(outputs_numpy)), outputs_numpy.argmax(1)] = 1
out = out.tolist()
results = save_results(out, targets, "pure_model-bubble_noise_results.csv")
results["loss"].mean()

100%|██████████| 188/188 [01:17<00:00,  2.41it/s]


0.1390056666666667

### Loss on pizza-noise images

In [41]:
test_dataset = ImageDataset(
    path=main_dir, parameters=DATASETS["pizza"]["test"], transform=transform
)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True, num_workers=0)

outputs, targets = test(test_loader)

outputs_numpy = np.array(outputs)
out = np.zeros_like(outputs_numpy)
out[np.arange(len(outputs_numpy)), outputs_numpy.argmax(1)] = 1
out = out.tolist()
results = save_results(out, targets, "pure_model-pizza_noise_results.csv")
results["loss"].mean()

100%|██████████| 188/188 [01:16<00:00,  2.45it/s]


0.15455699999999997

## Average Model - model trained using only average-noise images. ##

In [43]:
train_dataset = ImageDataset(
    path=main_dir, parameters=DATASETS["average"]["train"], transform=transform
)
valid_dataset = ImageDataset(
    path=main_dir, parameters=DATASETS["average"]["valid"], transform=transform
)
test_dataset = ImageDataset(
    path=main_dir, parameters=DATASETS["average"]["test"], transform=transform
)

In [44]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True, num_workers=0)

In [45]:
model = models.resnet18(pretrained=True)

classifier = nn.Sequential(
    nn.Linear(512, 512), nn.ReLU(), nn.Dropout(0.3), nn.ReLU(), nn.Linear(512, 1000)
)

model.fc = classifier

In [46]:
optimizer = torch.optim.AdamW(params=model.parameters(), lr=lr, eps=1e-8)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, patience=1, factor=0.9, min_lr=1e-5
)

In [47]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [48]:
min_valid_loss = np.inf
history = {"train_losses": [], "valid_losses": []}

model.cuda()

for epoch in tqdm(range(epochs)):
    model.train()
    train_batch_losses = []
    for data, labels in train_loader:
        model.zero_grad()
        optimizer.zero_grad()
        images = data.cuda()
        targets = labels.cuda()
        outputs = model(images)

        loss = loss_fn(outputs, targets)

        # optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_batch_losses.append(loss.item())
    train_loss = np.sum(train_batch_losses) / len(train_batch_losses)
    history["train_losses"].append(train_loss)

    model.eval()
    fin_targets = []
    fin_outputs = []
    valid_batch_losses = []
    with torch.no_grad():
        for data, labels in valid_loader:
            images = data.cuda()
            targets = labels.cuda()
            outputs = model(images)

            loss = loss_fn(outputs, targets)
            valid_batch_losses.append(loss.item())
        valid_loss = np.sum(valid_batch_losses) / len(valid_batch_losses)
        history["valid_losses"].append(valid_loss)

    if min_valid_loss > valid_loss:
        torch.save(model.state_dict(), "best_model_average.pth")
        min_valid_loss = valid_loss

    print(
        f"Epoch {epoch} \t\t Training Loss: {train_loss} \t\t Validation Loss: {valid_loss}"
    )
    scheduler.step(valid_loss)

    torch.save(model.state_dict(), "final_model_average.pth")

  0%|          | 0/5 [00:00<?, ?it/s]

 20%|██        | 1/5 [04:37<18:29, 277.47s/it]

Epoch 0 		 Training Loss: 0.055831491474317166 		 Validation Loss: 0.008047563302754137


 40%|████      | 2/5 [09:04<13:33, 271.15s/it]

Epoch 1 		 Training Loss: 0.008679741690952479 		 Validation Loss: 0.008006491439814266


...

Epoch 2 		 Training Loss: 0.008601821369374842 		 Validation Loss: 0.00796301742230548


 80%|████████  | 4/5 [17:27<04:17, 257.71s/it]

Epoch 3 		 Training Loss: 0.00850296359768627 		 Validation Loss: 0.007885362258950748


100%|██████████| 5/5 [21:31<00:00, 258.38s/it]

Epoch 4 		 Training Loss: 0.008256886542997083 		 Validation Loss: 0.007529806516100724


In [49]:
def test(test_loader):
    model.eval()
    fin_targets = []
    fin_outputs = []
    with torch.no_grad():
        for data, labels in tqdm(test_loader):
            images = data.cuda()
            targets = labels.cuda()
            outputs = model(images)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [50]:
def cyclic_MAE_test(output, target):
    return min(
        abs(output - target), min(abs(output - target + 1), abs(output - target - 1))
    )

In [51]:
def save_results(outputs, targets, filename: str):
    epsilons = []
    predicted = []
    differences = []
    for out, tar in zip(outputs, targets):
        epsilons.append(tar.index(1) / 1000)
        predicted.append(out.index(1) / 1000)
        differences.append(cyclic_MAE_test(out.index(1) / 1000, tar.index(1) / 1000))
    results_df = pd.DataFrame(
        list(zip(epsilons, predicted, differences)),
        columns=["epsilon", "predicted", "loss"],
    )
    results_df.to_csv(filename, index=False)
    return results_df

### Loss on pure images ###

In [52]:
test_dataset = ImageDataset(
    path=main_dir, parameters=DATASETS["pure"]["test"], transform=transform
)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True, num_workers=0)

outputs, targets = test(test_loader)

outputs_numpy = np.array(outputs)
out = np.zeros_like(outputs_numpy)
out[np.arange(len(outputs_numpy)), outputs_numpy.argmax(1)] = 1
out = out.tolist()
results = save_results(out, targets, "pure_model-pure_noise_results.csv")
results["loss"].mean()

100%|██████████| 188/188 [01:16<00:00,  2.45it/s]


0.24351566666666669

### Loss on average-noise images

In [53]:
test_dataset = ImageDataset(
    path=main_dir, parameters=DATASETS["average"]["test"], transform=transform
)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True, num_workers=0)

outputs, targets = test(test_loader)

outputs_numpy = np.array(outputs)
out = np.zeros_like(outputs_numpy)
out[np.arange(len(outputs_numpy)), outputs_numpy.argmax(1)] = 1
out = out.tolist()
results = save_results(out, targets, "average_model-average_noise_results.csv")
results["loss"].mean()

100%|██████████| 188/188 [01:19<00:00,  2.36it/s]


0.06025333333333331

### Loss on all-noise images

In [54]:
test_dataset = ImageDataset(
    path=main_dir, parameters=DATASETS["all"]["test"], transform=transform
)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True, num_workers=0)

outputs, targets = test(test_loader)

outputs_numpy = np.array(outputs)
out = np.zeros_like(outputs_numpy)
out[np.arange(len(outputs_numpy)), outputs_numpy.argmax(1)] = 1
out = out.tolist()
results = save_results(out, targets, "average_model-all_noise_results.csv")
results["loss"].mean()

100%|██████████| 188/188 [01:22<00:00,  2.28it/s]


0.11756866666666667

### Loss on blackbox-noise images

In [55]:
test_dataset = ImageDataset(
    path=main_dir, parameters=DATASETS["blackbox"]["test"], transform=transform
)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True, num_workers=0)

outputs, targets = test(test_loader)

outputs_numpy = np.array(outputs)
out = np.zeros_like(outputs_numpy)
out[np.arange(len(outputs_numpy)), outputs_numpy.argmax(1)] = 1
out = out.tolist()
results = save_results(out, targets, "average_model-blackbox_noise_results.csv")
results["loss"].mean()

100%|██████████| 188/188 [01:20<00:00,  2.33it/s]


0.24361000000000002

### Loss on bubble-noise images

In [56]:
test_dataset = ImageDataset(
    path=main_dir, parameters=DATASETS["bubble"]["test"], transform=transform
)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True, num_workers=0)

outputs, targets = test(test_loader)

outputs_numpy = np.array(outputs)
out = np.zeros_like(outputs_numpy)
out[np.arange(len(outputs_numpy)), outputs_numpy.argmax(1)] = 1
out = out.tolist()
results = save_results(out, targets, "average_model-bubble_noise_results.csv")
results["loss"].mean()

100%|██████████| 188/188 [01:18<00:00,  2.40it/s]


0.25689866666666666

### Loss on pizza-noise images

In [57]:
test_dataset = ImageDataset(
    path=main_dir, parameters=DATASETS["pizza"]["test"], transform=transform
)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True, num_workers=0)

outputs, targets = test(test_loader)

outputs_numpy = np.array(outputs)
out = np.zeros_like(outputs_numpy)
out[np.arange(len(outputs_numpy)), outputs_numpy.argmax(1)] = 1
out = out.tolist()
results = save_results(out, targets, "average_model-pizza_noise_results.csv")
results["loss"].mean()

100%|██████████| 188/188 [01:19<00:00,  2.38it/s]


0.2502413333333333

## Theoretically best model - model trained on all training sets together ##

In [25]:
best_train = (
    pd.concat(
        [
            DATASETS["pure"]["train"],
            DATASETS["average"]["train"],
            DATASETS["all"]["train"],
            DATASETS["blackbox"]["train"],
            DATASETS["bubble"]["train"],
            DATASETS["pizza"]["train"],
        ]
    )
    .sample(frac=1, random_state=12)
    .reset_index(drop=True)
)

best_valid = (
    pd.concat(
        [
            DATASETS["pure"]["valid"],
            DATASETS["average"]["valid"],
            DATASETS["all"]["valid"],
            DATASETS["blackbox"]["valid"],
            DATASETS["bubble"]["valid"],
            DATASETS["pizza"]["valid"],
        ]
    )
    .sample(frac=1, random_state=12)
    .reset_index(drop=True)
)

In [26]:
train_dataset = ImageDataset(path=main_dir, parameters=best_train, transform=transform)
valid_dataset = ImageDataset(path=main_dir, parameters=best_valid, transform=transform)

In [27]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=True, num_workers=0)

In [28]:
model = models.resnet18(pretrained=True)

classifier = nn.Sequential(
    nn.Linear(512, 512), nn.ReLU(), nn.Dropout(0.3), nn.ReLU(), nn.Linear(512, 1000)
)

model.fc = classifier

/home/ute/users/danielj/topography-surface/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ute/users/danielj/topography-surface/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [29]:
optimizer = torch.optim.AdamW(params=model.parameters(), lr=lr, eps=1e-8)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, patience=1, factor=0.9, min_lr=1e-5
)

In [30]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [31]:
epochs = 10

In [26]:
min_valid_loss = np.inf
history = {"train_losses": [], "valid_losses": []}

model.cuda()

for epoch in range(epochs):
    model.train()
    train_batch_losses = []
    for data, labels in tqdm(train_loader):
        images = data.cuda()
        targets = labels.cuda()
        outputs = model(images)

        loss = loss_fn(outputs, targets)

        # optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_batch_losses.append(loss.item())
    train_loss = np.sum(train_batch_losses) / len(train_batch_losses)
    history["train_losses"].append(train_loss)

    model.eval()
    fin_targets = []
    fin_outputs = []
    valid_batch_losses = []
    with torch.no_grad():
        for data, labels in valid_loader:
            images = data.cuda()
            targets = labels.cuda()
            outputs = model(images)

            loss = loss_fn(outputs, targets)
            valid_batch_losses.append(loss.item())
        valid_loss = np.sum(valid_batch_losses) / len(valid_batch_losses)
        history["valid_losses"].append(valid_loss)

    if min_valid_loss > valid_loss:
        # torch.save(model.state_dict(), 'best_model_average.pth')
        min_valid_loss = valid_loss

    print(
        f"Epoch {epoch} \t\t Training Loss: {train_loss} \t\t Validation Loss: {valid_loss}"
    )
    scheduler.step(valid_loss)

    torch.save(model.state_dict(), "final_model_best.pth")

  0%|          | 0/1125 [00:00<?, ?it/s]

  5%|▍         | 51/1125 [00:36<12:39,  1.41it/s]


KeyboardInterrupt: 

In [ ]:
def test(test_loader):
    model.eval()
    fin_targets = []
    fin_outputs = []
    with torch.no_grad():
        for data, labels in tqdm(test_loader):
            images = data.cuda()
            targets = labels.cuda()
            outputs = model(images)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [ ]:
def cyclic_MAE_test(output, target):
    return min(
        abs(output - target), min(abs(output - target + 1), abs(output - target - 1))
    )

In [ ]:
def save_results(outputs, targets, filename: str):
    epsilons = []
    predicted = []
    differences = []
    for out, tar in zip(outputs, targets):
        epsilons.append(tar.index(1) / 1000)
        predicted.append(out.index(1) / 1000)
        differences.append(cyclic_MAE_test(out.index(1) / 1000, tar.index(1) / 1000))
    results_df = pd.DataFrame(
        list(zip(epsilons, predicted, differences)),
        columns=["epsilon", "predicted", "loss"],
    )
    results_df.to_csv(filename, index=False)
    return results_df

### Loss on pure images ###

In [ ]:
test_dataset = ImageDataset(
    path=main_dir, parameters=DATASETS["pure"]["test"], transform=transform
)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True, num_workers=0)

outputs, targets = test(test_loader)

outputs_numpy = np.array(outputs)
out = np.zeros_like(outputs_numpy)
out[np.arange(len(outputs_numpy)), outputs_numpy.argmax(1)] = 1
out = out.tolist()
results = save_results(out, targets, "pure_model-pure_noise_results.csv")
results["loss"].mean()

100%|██████████| 188/188 [01:16<00:00,  2.45it/s]


0.24351566666666669

### Loss on average-noise images

In [ ]:
test_dataset = ImageDataset(
    path=main_dir, parameters=DATASETS["average"]["test"], transform=transform
)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True, num_workers=0)

outputs, targets = test(test_loader)

outputs_numpy = np.array(outputs)
out = np.zeros_like(outputs_numpy)
out[np.arange(len(outputs_numpy)), outputs_numpy.argmax(1)] = 1
out = out.tolist()
results = save_results(out, targets, "average_model-average_noise_results.csv")
results["loss"].mean()

100%|██████████| 188/188 [01:19<00:00,  2.36it/s]


0.06025333333333331

### Loss on all-noise images

In [ ]:
test_dataset = ImageDataset(
    path=main_dir, parameters=DATASETS["all"]["test"], transform=transform
)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True, num_workers=0)

outputs, targets = test(test_loader)

outputs_numpy = np.array(outputs)
out = np.zeros_like(outputs_numpy)
out[np.arange(len(outputs_numpy)), outputs_numpy.argmax(1)] = 1
out = out.tolist()
results = save_results(out, targets, "average_model-all_noise_results.csv")
results["loss"].mean()

100%|██████████| 188/188 [01:22<00:00,  2.28it/s]


0.11756866666666667

### Loss on blackbox-noise images

In [ ]:
test_dataset = ImageDataset(
    path=main_dir, parameters=DATASETS["blackbox"]["test"], transform=transform
)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True, num_workers=0)

outputs, targets = test(test_loader)

outputs_numpy = np.array(outputs)
out = np.zeros_like(outputs_numpy)
out[np.arange(len(outputs_numpy)), outputs_numpy.argmax(1)] = 1
out = out.tolist()
results = save_results(out, targets, "average_model-blackbox_noise_results.csv")
results["loss"].mean()

100%|██████████| 188/188 [01:20<00:00,  2.33it/s]


0.24361000000000002

### Loss on bubble-noise images

In [ ]:
test_dataset = ImageDataset(
    path=main_dir, parameters=DATASETS["bubble"]["test"], transform=transform
)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True, num_workers=0)

outputs, targets = test(test_loader)

outputs_numpy = np.array(outputs)
out = np.zeros_like(outputs_numpy)
out[np.arange(len(outputs_numpy)), outputs_numpy.argmax(1)] = 1
out = out.tolist()
results = save_results(out, targets, "average_model-bubble_noise_results.csv")
results["loss"].mean()

100%|██████████| 188/188 [01:18<00:00,  2.40it/s]


0.25689866666666666

### Loss on pizza-noise images

In [ ]:
test_dataset = ImageDataset(
    path=main_dir, parameters=DATASETS["pizza"]["test"], transform=transform
)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True, num_workers=0)

outputs, targets = test(test_loader)

outputs_numpy = np.array(outputs)
out = np.zeros_like(outputs_numpy)
out[np.arange(len(outputs_numpy)), outputs_numpy.argmax(1)] = 1
out = out.tolist()
results = save_results(out, targets, "average_model-pizza_noise_results.csv")
results["loss"].mean()

100%|██████████| 188/188 [01:19<00:00,  2.38it/s]


0.2502413333333333

# Possibly the best model trained on all data together #